In [ ]:
from IPython.display import display, clear_output
import utils
import ipywidgets as ipw
import widgets
import datetime

In [ ]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.read_json("eln_config.json")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])
SAMPLE_PREPARATION_SAMPLE_TYPES = [object_info["openbis_object_type"] for _, object_info in CONFIG["objects"].items() if object_info["object_type"] == "sample_preparation"]
SLABS_TYPES = [object_info["openbis_object_type"] for _, object_info in CONFIG["objects"].items() if object_info["object_type"] == "slab"]

sample_prep_selector = widgets.SamplePreparationSelectionWidget()

sample_prep_accordion = ipw.Accordion(children = [])

experiment_selector = widgets.ExperimentSelectionWidget()
experiment_selector.load_dropdown_box()

sample_selector = widgets.SampleSelectionWidget()
sample_selector.load_dropdown_box()

instrument_selector = widgets.InstrumentSelectionWidget()
instrument_selector.load_dropdown_box()

sample_out_name_textbox = utils.Text(
    description = "Name", disabled = False, layout = ipw.Layout(width = '400px'), 
    placeholder = f"Write sample name here...", style = {'description_width': "110px"}
)

increase_buttons_size = utils.HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))
create_button = utils.Button(
    description = '', disabled = False, button_style = '', tooltip = 'Save', 
    icon = 'save', layout = ipw.Layout(width = '100px', height = '50px')
)
quit_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)
save_close_buttons_hbox = ipw.HBox([create_button, quit_button])

In [ ]:
def load_accordion_widgets(change):
    accordion_options = []
    
    for i, task in enumerate(sample_prep_selector.value):
        
        task_widgets = []
        
        if CONFIG["objects"][task]["uses_substance"]:
            substance_selection_widget = widgets.SubstanceSelectionWidget()
            substance_selection_widget.load_dropdown_box()
            substance_selection_widget.dropdown_boxes.children[0].observe(substance_selection_widget.load_substance_metadata, names = 'value')
            task_widgets.append(substance_selection_widget)
        
        properties_widgets = widgets.ObjectPropertiesWidgets(task)
        task_widgets.append(properties_widgets)
        
        support_files = ipw.FileUpload(multiple = True)
        task_widgets.append(support_files)
        
        sample_prep_accordion.set_title(i, task)
        accordion_options.append(ipw.VBox(task_widgets))
        
    sample_prep_accordion.children = accordion_options

def load_sample_metadata(change):
    if sample_selector.dropdown_boxes.children[0].value == -1:
        experiment_selector.dropdown.value = -1
        instrument_selector.dropdown_boxes.children[0].value = -1
        sample_selector.details_textbox.value = ''
        sample_out_name_textbox.value = ''
        return
    
    sample_object = OPENBIS_SESSION.get_object(sample_selector.dropdown_boxes.children[0].value)
    sample_parents_metadata = utils.get_openbis_parents_recursive(OPENBIS_SESSION, sample_object, [])
    last_sample_preparation = None
    sample_strings = {"sample_preparations": [], "materials": [], "sample_preparation_datetimes": []}
    number_parents = len(sample_parents_metadata)
    parent_idx = 0
    while parent_idx < number_parents:
        parent_metadata = sample_parents_metadata[parent_idx]
        if parent_metadata[0] == "DEPOSITION":
            deposition_object = OPENBIS_SESSION.get_object(parent_metadata[1])
            
            # Get substance used in the specific deposition
            substance_metadata = []
            for parent_id in deposition_object.parents:
                deposition_parent_object = OPENBIS_SESSION.get_object(parent_id)
                if deposition_parent_object.type == "SUBSTANCE":
                    deposition_parent_object_metadata = deposition_parent_object.props.all()
                    substance_metadata = [deposition_parent_object_metadata["$name"], deposition_parent_object.permId]
                    
            if substance_metadata: # If deposition does not contain any substance, the app must not try to display it
                sample_metadata_string = f"> {parent_metadata[0]} ({parent_metadata[3]}, {parent_metadata[1]}, {parent_metadata[2]}) [{substance_metadata[0]} ({substance_metadata[1]})]"
            else:
                sample_metadata_string = f"> {parent_metadata[0]} ({parent_metadata[3]}, {parent_metadata[1]}, {parent_metadata[2]})"
            parent_idx += 1
        else:
            sample_metadata_string = f"> {parent_metadata[0]} ({parent_metadata[3]}, {parent_metadata[1]}, {parent_metadata[2]})"
        
        if parent_metadata[0] in SAMPLE_PREPARATION_SAMPLE_TYPES:
            if sample_metadata_string not in sample_strings["sample_preparations"]:
                sample_strings["sample_preparations"].append(sample_metadata_string)
                sample_strings["sample_preparation_datetimes"].append(parent_metadata[2])
            # Get the last sample preparation method performed on the sample in order to search the correct experiment where the sample is being used.
            if last_sample_preparation is None:
                last_sample_preparation = parent_metadata[1]
                
        elif parent_metadata[0] in SLABS_TYPES:
            if sample_metadata_string not in sample_strings["materials"]:
                sample_strings["materials"].append(sample_metadata_string)
            
        parent_idx += 1
    
    # Parse the datetime strings and zip them with sample preparations and materials
    parsed_datetimes = [datetime.datetime.strptime(dt, "%Y-%m-%d %H:%M:%S") for dt in sample_strings["sample_preparation_datetimes"]]
    zipped_lists = list(zip(parsed_datetimes, sample_strings["sample_preparations"]))

    # Sort by the datetime
    sorted_zipped_lists = sorted(zipped_lists, key=lambda x: x[0], reverse = True)

    # Unpack the sorted values back into sample_strings
    _, sample_strings["sample_preparations"] = zip(*sorted_zipped_lists)
    
    sample_metadata_string = (f"PermId: {sample_object.attrs.permId}\nMaterial:\n" +
                            "\n".join(sample_strings["materials"]) + 
                            "\nProcesses:\n" + 
                            "\n".join(sample_strings["sample_preparations"]))

    if last_sample_preparation:
        last_sample_preparation_object = OPENBIS_SESSION.get_object(last_sample_preparation)
        # Automatically select the experiment where the last sample preparation task was saved
        last_sample_preparation_experiment = OPENBIS_SESSION.get_experiment(last_sample_preparation_object.attrs.experiment)
        # Notify the user in case the experiment changed
        if experiment_selector.dropdown.value != last_sample_preparation_experiment.permId and experiment_selector.dropdown.value != -1:
            dropdown_options_dict = {key: val for val, key in experiment_selector.dropdown.options}
            previous_experiment_name = dropdown_options_dict.get(experiment_selector.dropdown.value)
            new_experiment_name = dropdown_options_dict.get(last_sample_preparation_experiment.permId)
            display(utils.Javascript(data = f"alert('{f'Experiment was changed from {previous_experiment_name} to {new_experiment_name}!'}')"))
        experiment_selector.dropdown.value =  last_sample_preparation_experiment.permId
        
        # Automatically select the instrument used in the last sample preparation task
        for parent in last_sample_preparation_object.parents:
            parent_object = OPENBIS_SESSION.get_object(parent)
            if parent_object.type == "INSTRUMENT":
                instrument_selector.dropdown_boxes.children[0].value = parent_object.permId
    
    sample_selector.details_textbox.value = sample_metadata_string
    sample_out_name = sample_object.props['$name']
    
    sample_out_name = ""
    for i, task in enumerate(sample_prep_selector.value):
        task_name = sample_prep_accordion.children[i].children[0].properties_widgets['$name'].value
        if task_name:
            sample_out_name = f"{sample_out_name}_{task_name}"
            
    sample_out_name_textbox.value = sample_out_name
    

In [ ]:
sample_prep_selector.observe(load_accordion_widgets, names = 'value')

# Sample Preparation

In [ ]:
# Widgets
display(sample_prep_selector)

display(utils.Markdown(data = "## Select experiment, sample, and instrument"))
display(experiment_selector, sample_selector, instrument_selector)

display(utils.Markdown(data = "## Sample Preparation Tasks"))
display(sample_prep_accordion)

display(utils.Markdown(data = "## Save Sample"))
display(sample_out_name_textbox)
display(save_close_buttons_hbox)
display(increase_buttons_size)

sample_selector.dropdown_boxes.children[0].observe(load_sample_metadata, names = 'value')
# app_widgets.create_button.on_click(app_widgets.create_sample_preparation_action)
# app_widgets.quit_button.on_click(app_widgets.close_notebook)

# for i, task in enumerate(app_widgets.sample_prep_selector.options):
#     app_widgets.object_widgets[task]["$name"].observe(app_widgets.update_text, names = 'value')